# Real Estate Data Generataion

# 1. Data Generation Prompt Build

From some House Match Website [House Match](https://search.housematch.com/) e.g. I collect which feature should a real estate data has.

```
4 Bedrooms 2 Bathrooms 2,018 Size sqft 9,583.2 Lot sqft
Single Family Detached • Built in 1959 • $180/SqFt • 1 day on site
Fabulous pool home on a large fenced lot, with four bedrooms and two bathrooms with ceramic tile floors throughout the living space and luxury vinyl in the bedrooms. Upgrades throughout include a recently updated hall bath, remodeled kitchen, and new floors; the pool was resurfaced in 2018; the roof and the electrical were updated in 2018; interior paint was updated in 2020; a new range, dishwasher, microwave, and pool pavers were installed in 2023; the exterior was repainted in 2024. Huge backyard, new vinyl fence (2023), huge pool, and plenty of entertaining areas. All bedrooms have ceiling fans. The Great Room leads to the kitchen and dining room with sliding glass doors that lead to the pool area. The seller is offering a home warranty. No HOA, No CDD, and low taxes make this a perfect home for any buyer. It is located near I-75, minutes to retail, shopping, The Grove, Krates at the Grove, Tampa Premium Outlet Mall, Medical, and access to Downtown Tampa, Orlando, and the sunny beaches.
```
based on this sample and my own experience, I think a House Match estate data for rental should have:

- Location
- House type (SFR, Mansion e.g., if mansion which level)
- Square & Layout (LDK?)
- Rental Price
- Building info
    -  building year
    -  structure (wood, concrete, steel e.g.)
- Living Related
    - Nearest SuperMarket
    - Nearest SubwayStation

- Some Detail Descrption(All furniture, Air Conditionar e.g.)

In [11]:
import os
import dotenv

if (envfiles:= dotenv.find_dotenv() )!="":
    dotenv.load_dotenv(envfiles)
else:
    print(".env file not found! Please set OPENAPI KEY first")

I would divide the prompt generate into two steps, as the Location, House type, e.g. is some short text or string that represent some class, it can be seen as a features directly, but detail descrption is a more complicated documentary that merge all this information together but also provide new from other point of view.

so I would first generate a dict like data that can be analysied by Pydantic Prompt, and then based on the generated basic features generated a detail description, and at last merge all these together.

the prompt itself is already test on chatgpt web application and refined by prompt engineering in GPTs, as the prompt may run many times, I would not want to let it use too many tokens.

## 1.1 generate basic house information pipeline

In [41]:
# House Attribute Model
from typing import List
from langchain_core.pydantic_v1 import BaseModel, Field

class HouseType(BaseModel):
    type: str = Field(description="type of this house, like apartment ,condominium e.g.")
    square: str = Field(description="the square of this hosue.")
    layout: str = Field(description="layout of this house, like 1LDK, 2LDK, 1DK e.g.")
    layout_details: str = Field(description="""details about layout of this house, 
                      like how much square of living room e.g.""")

class BuildInfo(BaseModel):
    year: int = Field(description="""when the house built,
                      should smaller than 2024 but bigger than 1980.""")
    structure: str = Field(description="""the structure of this house,
                           one of the following selection: wood, steel, concrete.""")

class LivingInfo(BaseModel):
    nearest_supermarket: str = Field(description="""how long and by which method
                                     will it take to go to nearest supermarket.
                                     The method would be by bus or by walk""")
    nearest_subway: str = Field(description="""how long and by which method
                                     will it take to go to nearest subwaystation.
                                     The method would be by bus or by walk""")

class House(BaseModel):
    location: str = Field(description="""
                          the location of this house, include two attribute,
                          city and ward, city selection: Tokyo, Yokohama, Chiba,
                          ward is a ward in that city.
                          """)
    type: HouseType
    price: str = Field(description="""the money should be paid to
                       rent this house monthly, in dollars,
                       basically it would be $800 for 1ldk in Tokyo
                       with slight change based on the ward and square""")
    built_info: BuildInfo
    live_info: LivingInfo

class HouseList(BaseModel):
    house_list: List[House]

In [42]:
from langchain.prompts import PromptTemplate
from langchain_community.chat_models import ChatOpenAI
from langchain.output_parsers import PydanticOutputParser

model = ChatOpenAI()

parser = PydanticOutputParser(pydantic_object=HouseList)
parser_prompt = parser.get_format_instructions()

# This Prompt would not generate exact num_of_data, because ChatGPT may
# not remember how much data he need to generate  when N is big,
# N around 10 works fine, but when N comes to 20, it would generate
# 6 ~ 25 data at a time.
prompt = PromptTemplate(
    template="""Generate {num_of_data} house information for rental in house_list.
    Return the final result as JSON object direclty.\n{format_instructions}\n""",
    input_variables=["num_of_data"], 
    input_types={"num_of_data": int},
    partial_variables={"format_instructions": parser.get_format_instructions()},
)

chain = prompt | model | parser
# test
output = chain.invoke({"num_of_data":2})

In [43]:
output

HouseList(house_list=[House(location='Tokyo, Shibuya', type=HouseType(type='Apartment', square='50 sqm', layout='1LDK', layout_details='20 sqm living room'), price='$1000', built_info=BuildInfo(year=2010, structure='Concrete'), live_info=LivingInfo(nearest_supermarket='10 minutes by walk', nearest_subway='5 minutes by bus')), House(location='Yokohama, Kanagawa', type=HouseType(type='Condominium', square='70 sqm', layout='2LDK', layout_details='25 sqm living room'), price='$1200', built_info=BuildInfo(year=2015, structure='Steel'), live_info=LivingInfo(nearest_supermarket='15 minutes by walk', nearest_subway='10 minutes by bus'))])

In [ ]:
# create 200 data, may take around 30 minutes
generated_data = []
for i in range(12):
    output = chain.invoke({"num_of_data":20})
    generated_data.append(output.house_list)

In [54]:
import numpy as np
flatten_data: List[House] = np.concatenate(generated_data).ravel().tolist()

In [74]:
import json
import pandas as pd
df = None

for house_obj in flatten_data:
    raw_attr = json.loads(house_obj.json())
    house_attr = raw_attr.copy()
    # normalize json object
    for key, val in raw_attr.items():
        if isinstance(val, dict):
            val = raw_attr[key]
            house_attr.pop(key)
            for sub_key, sub_val in val.items():
                house_attr[key+'_'+sub_key] = sub_val
        else:
            pass
    house_attr['index'] = [0]
    row = pd.DataFrame.from_dict(house_attr)
    if df is None:
        df = row
    else:
        df = pd.concat([df, row], axis=0)
df = df.reset_index(drop=True)
df = df.drop(columns=["index"])
df.to_csv("chatgpt_house_match_data.csv", index=False)

In [75]:
df.head()

,location,price,type_type,type_square,type_layout,type_layout_details,built_info_year,built_info_structure,live_info_nearest_supermarket,live_info_nearest_subway
0,"Tokyo, Shibuya",$800,apartment,50 sqm,1LDK,20 sqm living room,2005,concrete,10 minutes by walk,5 minutes by bus
1,"Yokohama, Minato",$1000,condominium,80 sqm,2LDK,30 sqm living room,1998,steel,15 minutes by walk,10 minutes by bus
2,"Chiba, Urayasu",$1200,house,120 sqm,3LDK,40 sqm living room,2010,wood,5 minutes by walk,20 minutes by bus
3,"Tokyo, Shinjuku",$900,apartment,60 sqm,1DK,25 sqm living room,1990,concrete,20 minutes by walk,15 minutes by bus
4,"Yokohama, Nishi",$1100,condominium,90 sqm,2LDK,35 sqm living room,2002,steel,10 minutes by walk,10 minutes by bus


# 1.2 generate description pipeline